In [21]:
import random
import string
import math

Frase='Me llamo Imanol'.lower
numPoblacion=5
generaciones=10
pMutacion=0.3
resumen=2
muestra=5

In [25]:
def binario (cadena):
#La funcion recibe la cadena y devuelve un array.
#Cada posicion del array representa una letra de la cadena

    bits=[]

    for letra in bytearray(cadena, "utf8"):
        binario=bin(letra)
        bits.append(binario)
        
    return bits


def natural (binario):
    #La funcion recibe el array con todos los caracteres en binario
    #devuelve la cadena en lengueje natural
    texto=''

    for i in range(len(binario)):
        texto+=chr(int(binario[i],2))

    return texto


def coincidencias(cadena,original):
    num=0
    for i in range(len(original)):
        if cadena[i]==original[0]:
            num+=1
    return num


def fitness(cadena, original, coincidencias):
    return math.exp(coincidencias - len(original)) - math.exp(-len(original))


In [17]:
poblacion=[]
fitnes=[]
letras=string.ascii_lowercase

#Poblacion aleatoria
for i in range(numPoblacion):
    poblacion.append(''.join(random.choice(letras) for j in range(len(Frase))))

#Calculamos el fitness
for i in range(numPoblacion):
    coincide=coincidencias(poblacion[i], Frase)
    fitnes= fitness(poblacion[i], Frase, coincide)
    

Me llamo imanol
